In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib
.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *

DATA_TRAIN_PATH = '../../../Data-Project1/train.csv' # TODO: add a file Data-Project1 with the train data 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
from helpers import *

# Standardize the input data
stx, mean_stx, std_x = standardize(tX)

In [4]:
# TODO: Get rid of the -999 in the data

## Do your thing crazy machine learning thing here :) ...

## Linear regression using gradient descent

In [4]:
from costs import *

def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y - tx @ w
    return -(tx.T @ e)/len(y)

def least_squares_GD(y, tx, gamma, max_iters):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    w = np.zeros(tx.shape[1])
    ws = [np.copy(w)]
    losses = []
    for n_iter in range(max_iters):
        # compute gradient with mse
        grad = compute_gradient(y, tx, w)
        
        # update w
        w = w - (gamma*grad)
        
        # calculate the cost with mse
        loss = compute_loss(y, tx, w)
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))
    print("Optimal weights: {w}".format(w=ws[tx.shape[1]]))
    return losses, ws

gamma = 0.1
max_iters = 100
least_squares_GD(y, stx, gamma, max_iters)

Gradient Descent(0/99): loss=0.45383148038975835
Gradient Descent(1/99): loss=0.4310979587324953
Gradient Descent(2/99): loss=0.41538321133630235
Gradient Descent(3/99): loss=0.4039178092406891
Gradient Descent(4/99): loss=0.39532813034051306
Gradient Descent(5/99): loss=0.38874321747505025
Gradient Descent(6/99): loss=0.38358837180705296
Gradient Descent(7/99): loss=0.379476061272276
Gradient Descent(8/99): loss=0.3761393708840319
Gradient Descent(9/99): loss=0.3733904841335668
Gradient Descent(10/99): loss=0.37109444248742285
Gradient Descent(11/99): loss=0.36915232673600373
Gradient Descent(12/99): loss=0.36749030408999545
Gradient Descent(13/99): loss=0.3660523676818952
Gradient Descent(14/99): loss=0.3647954306311018
Gradient Descent(15/99): loss=0.36368594403191645
Gradient Descent(16/99): loss=0.36269751783569354
Gradient Descent(17/99): loss=0.3618092138596624
Gradient Descent(18/99): loss=0.36100429801152073
Gradient Descent(19/99): loss=0.36026931252143907
Gradient Descent(20

([0.45383148038975835,
  0.43109795873249529,
  0.41538321133630235,
  0.4039178092406891,
  0.39532813034051306,
  0.38874321747505025,
  0.38358837180705296,
  0.37947606127227601,
  0.37613937088403188,
  0.37339048413356679,
  0.37109444248742285,
  0.36915232673600373,
  0.36749030408999545,
  0.3660523676818952,
  0.36479543063110181,
  0.36368594403191645,
  0.36269751783569354,
  0.36180921385966242,
  0.36100429801152073,
  0.36026931252143907,
  0.35959337558073295,
  0.35896764564279676,
  0.35838490704976517,
  0.35783924647103194,
  0.35732579826177557,
  0.35684054275515564,
  0.35638014562352799,
  0.355941829374677,
  0.35552327017024388,
  0.35512251471449718,
  0.35473791312763231,
  0.35436806460048842,
  0.35401177330353462,
  0.35366801254572661,
  0.35333589558650863,
  0.35301465182435354,
  0.35270360733809031,
  0.35240216895793808,
  0.35210981120307544,
  0.35182606555041379,
  0.35155051160174078,
  0.35128276979876533,
  0.35102249540192199,
  0.35076937350

## Lienear regression using stochastic gradient descent

In [6]:
def least_squares_SGD(y, tx, gamma, max_iters):
    """Stochastic gradient descent algorithm."""
    # Define parameters to store w and loss
    w = np.zeros(tx.shape[1])
    ws = [np.copy(w)]
    losses = []
    batch_size = 50000
    
    for n_iter in range(max_iters):
        # compute gradient for each ini batch
        a = 0;
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size, num_batches=1):
            grad = compute_gradient(minibatch_y, minibatch_tx, w)
            
            # upgrade w
            w = w - (gamma*grad)

            # compute loss with mse
            loss = compute_loss(y, tx, w)

            # store w and loss
            ws.append(np.copy(w))
            losses.append(loss)
    print("Optimal weights: {w}\n\n Loss: {l}".format(w=ws[tx.shape[1]], l = loss))
    return losses, ws

least_squares_SGD(y, stx, gamma, max_iters)

Optimal weights: [ -3.02679029e-01   6.99275305e-02  -1.95117643e-01  -6.23107195e-02
   5.07896289e-02   5.03919395e-05   1.17350756e-01  -2.29781820e-03
   1.04870232e-01  -4.43452730e-02   2.96075516e-03  -8.71774250e-02
   1.18833108e-01  -5.93842074e-04   1.38855600e-01  -1.19567194e-03
  -2.74114337e-03   6.83553546e-02  -6.53962232e-04   2.18279312e-03
   4.03787267e-02   9.38125861e-04  -3.65489123e-02  -5.22712387e-02
   7.61286001e-03   7.37990341e-03   7.38515023e-03  -2.88575412e-03
  -7.76230746e-04  -7.92592771e-04  -4.36432149e-02]
 Loss: 0.34323002703586303


([0.45367851166532724,
  0.43126528328294356,
  0.41524439888253745,
  0.4038674692326627,
  0.39526224081182409,
  0.38866472352115067,
  0.38351695582484069,
  0.37948772764217203,
  0.37616127693786,
  0.37338776147928865,
  0.37108120479725221,
  0.36905673842888881,
  0.36737025151561903,
  0.36600811868451749,
  0.3648268670561956,
  0.36372132642783161,
  0.36275099534681154,
  0.36186252141768782,
  0.36100732174375622,
  0.36030436546886879,
  0.35960243678700532,
  0.35898717771165317,
  0.35843229969066359,
  0.35789086555791683,
  0.35735203808123744,
  0.35688129106487637,
  0.35641109230400075,
  0.35598514593259561,
  0.35557290643074946,
  0.35515400562252103,
  0.35476967539449411,
  0.35438247540998258,
  0.35401099260917712,
  0.35365763463870087,
  0.35332125699879757,
  0.35299555823173984,
  0.35270936138145498,
  0.35240042954584877,
  0.35209893395997804,
  0.35181479913641395,
  0.35156077379897338,
  0.35128418581869431,
  0.35103882655752988,
  0.350784053831

## Least squares regression using normal equations

In [9]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    w = np.linalg.solve(tx.T @ tx, tx.T @ y)
    return w, compute_loss(y, tx, w)

least_squares(y, stx)

(array([ -3.14664000e-01,   2.93788273e-02,  -2.52531475e-01,
         -2.54791124e-01,  -3.03696824e-02,  -1.40144708e+00,
          2.95701640e-01,  -1.07889472e+01,   2.67880862e-01,
         -2.44934968e-03,  -3.28831671e+02,  -1.82647888e-01,
          1.14039627e-01,   2.05045958e+01,   6.38861362e+01,
         -3.18961886e-04,  -1.80884297e-03,   6.29952758e+01,
         -4.48641562e-04,   1.54379300e-03,   1.21462700e-01,
          3.95268733e-04,  -6.33223476e-02,  -2.06747093e-01,
         -1.16655764e-01,   9.86256185e-02,   1.67907730e-01,
         -3.35146216e-02,  -2.98358673e+00,  -5.36388103e+00,
          2.78482426e+02]), 0.33944559848933226)

## Ridge regression using normal equations

In [14]:
def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    w = np.linalg.solve((tx.T @ tx) + lamb*np.eye(tx.shape[1]), tx.T @ y)
    return w, compute_loss(y, tx, w)

lamb = 0
ridge_regression(y, stx, lamb)

(array([ -3.14663371e-01,   2.93125707e-02,  -2.52737438e-01,
         -2.54984230e-01,  -2.92559673e-02,   4.96823451e+00,
          3.13152616e-01,  -6.89566875e+00,   2.67900384e-01,
         -2.89361421e-03,   1.35384707e-02,  -1.82703349e-01,
          1.14263154e-01,   5.42828941e+00,   1.89503445e-01,
         -5.52011068e-04,  -1.39538587e-03,   2.85464851e-01,
         -7.28335421e-04,   2.07994500e-03,   1.21499302e-01,
          7.90494978e-04,  -6.30764463e-02,  -2.05888805e-01,
         -1.24130087e-01,   1.50140837e-01,   1.23307021e-01,
         -1.05830737e-02,  -1.09246635e+00,  -2.48060268e+00,
         -8.73311339e-02]), 0.33953303992068157)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)